In [3]:
import requests
 
api_key = ""
filename = "raw_pdfs/주간농사정보 제26호(2025.7.7.~7.13.).pdf"
 
urls = "https://api.upstage.ai/v1/document-digitization"
headers = {"Authorization": f"Bearer {api_key}"}
files = {"document": open(filename, "rb")}
data = {"ocr": "force", "base64_encoding": "['table']", "model": "document-parse"}
response = requests.post(urls, headers=headers, files=files, data=data)
 
print(response.json())

{'api': '2.0', 'content': {'html': '<header id=\'0\' style=\'font-size:16px\'>농촌진흥청</header>\n<h1 id=\'1\' style=\'font-size:20px\'>제26호</h1>\n<br><h1 id=\'2\' style=\'font-size:22px\'>주간농사정보</h1>\n<h1 id=\'3\' style=\'font-size:18px\'>2025. 7. 7. ~ 7. 13.</h1>\n<footer id=\'4\' style=\'font-size:14px\'>농촌진흥청에서는 금주에 꼭 실천해야 할 주요 농업기술 정보를<br>농업인들에게 매주 신속하게 제공하고 있습니다</footer>\n<h1 id=\'5\' style=\'font-size:18px\'>목 차</h1>\n<p id=\'6\' data-category=\'paragraph\' style=\'font-size:14px\'>제1장 농업정보 1<br>제2장 벼 4<br>제3장 밭 작 물 7<br>제4장 채 소 · 9<br>제5장 과 수 · 13<br>제6장 화 훼 18<br>제7장 특용작물 21<br>제8장 축 산 23<br>제9장 양 봉 28</p>\n<h1 id=\'7\' style=\'font-size:18px\'>요 약</h1>\n<br><table id=\'8\' style=\'font-size:14px\'><thead></thead><tbody><tr><td>분야</td><td>핵심기술 및 정보</td></tr><tr><td>농업 정보</td><td>· (기상) 기온은 평년(23.0~24.4°C)보다 높겠고, 강수량은 평년(44.5~95.3mm)과 비슷하거나 적겠음 * 아열대 고기압의 가장자리에 위치하여 덥고 습하겠음 · (저수율) 68.1%(평년 58.4%의 116.6%) * 6. 30. 기준</td></tr><tr><td>벼</td><td>· (본답관리) 이삭 패기 전 40일부터 30일 사이에 중간물떼기 실

In [6]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.json().get("content")["html"], 'html.parser')

chunks = []
current_title = None
current_content = ""

for tag in soup.find_all(['h1', 'p', 'table']):
    text = tag.get_text(separator="\n").strip()
    
    # h1 중에서 큰 제목을 새 chunk의 기준으로 사용
    if tag.name == 'h1' and 'font-size' in tag.attrs.get('style', ''):
        font_size = int(tag['style'].split("font-size:")[1].replace("px", "").strip())

        if font_size >= 20:
            if current_title and current_content:
                chunks.append({"title": current_title, "content": current_content.strip()})
                current_content = ""

            current_title = text

        else:
            current_content += f"\n{text}"

    elif tag.name in ['p', 'table']:
        current_content += f"\n{text}"

# 마지막 chunk 저장
if current_title and current_content:
    chunks.append({"title": current_title, "content": current_content.strip()})


In [7]:
chunks

[{'title': '주간농사정보',
  'content': '2025. 7. 7. ~ 7. 13.\n목 차\n제1장 농업정보 1\n제2장 벼 4\n제3장 밭 작 물 7\n제4장 채 소 · 9\n제5장 과 수 · 13\n제6장 화 훼 18\n제7장 특용작물 21\n제8장 축 산 23\n제9장 양 봉 28\n요 약\n분야\n핵심기술 및 정보\n농업 정보\n· (기상) 기온은 평년(23.0~24.4°C)보다 높겠고, 강수량은 평년(44.5~95.3mm)과 비슷하거나 적겠음 * 아열대 고기압의 가장자리에 위치하여 덥고 습하겠음 · (저수율) 68.1%(평년 58.4%의 116.6%) * 6. 30. 기준\n벼\n· (본답관리) 이삭 패기 전 40일부터 30일 사이에 중간물떼기 실시 · (병해충방제) 먹노린재, 도열병 등 병해충 수시 예찰 및 적기 방제 · (집중호우 대비) 논·밭두렁, 제방, 배수로 등 사전점검 및 정비\n밭작물\n· (고온대비) 스프링클러 가동, 토양 피복으로 지온 상승 억제, 고온성 해충 발생 증가에 대비해 예찰 및 방제 실시 · (콩) 고온이 지속되면 파밤나방, 담배거세미나방이 밀도가 증가하므로 적용 약제 살포 · (참깨) 잘록병, 역병, 시들음병, 잎마름병 등 예방 위주 방제\n채소\n· (고추) 고온 대비 토양 수분 유지, 역병, 탄저병 등 병해충 방제, · (여름배추·무) 석회결핍 예방, 뿌리혹병, 무름병 등 약제 살포 · (시설채소) 고온 대비 환기팬 가동 및 차광, 병해충 방제 및 엽면시비 실시\n과수\n· (햇볕데임) 일 최고기온 31°C 이상 맑은 날 발생, 양광면 음광면에 비해 10°C 높음 - 적절한 가지 배치를 통한 일소피해 예방, 미세살수장치 가동 필요 · (장마·태풍대비) 배수로 정비, 경사지 과원 토양유실 방지, 방풍수방풍망 등 점검 - 침수된 과원 배수로 정비, 흙 앙금은 씻어주고 병든 과실은 제거 · (병해충 방제) 겹무늬썩음병, 갈색무늬병, 잿빛무늬병, 노균병 등 사전예방 약제 살포\n화훼\n· (심비디움) 8월까지

In [8]:
import boto3

# MinIO 설정
minio_endpoint = "http://artifact-store:9000"  # 도커 내부라면 'host.docker.internal:9000'
access_key = "minio"
secret_key = "miniostorage"

# S3 클라이언트 생성 (endpoint_url 반드시 지정)
s3_client = boto3.client(
    's3',
    endpoint_url=minio_endpoint,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
)


In [9]:
import time
import uuid

today = time.strftime('%Y-%m-%d')
doc_id = time.strftime('%Y-%m-%d_')+str(uuid.uuid4())[:10]

In [10]:
for i, chunk in enumerate(chunks):
    chunk["chunk_id"] = f"{doc_id}_{i:03}"
    chunk["date"] = today

In [11]:
import io 
import json 

buffer = io.BytesIO()
buffer.write(json.dumps(chunks, ensure_ascii=False).encode("utf-8"))
buffer.seek(0)

# 저장 파일명
filename = f"data/{doc_id}.json"

s3_client.upload_fileobj(buffer, "chunk", filename)
print("업로드 완료")

업로드 완료


In [12]:
raw_buffer = io.BytesIO() 


raw_buffer.write(json.dumps(response.json(), ensure_ascii=False).encode("utf-8"))
raw_buffer.seek(0)

raw_id= time.strftime('%Y-%m-%d_')+str(uuid.uuid4())[:10]

raw_files = f"raw_data/{raw_id}.json"
raw_files

s3_client.upload_fileobj(raw_buffer, "ocr-raw", raw_files)